# cours

## ré-importation (classes)

## formulas/formules

$$
init \left\{
\begin{array}{l}
P_1(1,1) = 0 \\
P_1(1,2) = P_{12} = 4 \\
P_1(1,3) = P_{13} = 6 \\
P_1(1,4) = P_1(1,5) = P_1(1,6) = \inf
\end{array}
\right.
$$

$$
P_2(1,2)=min(P_1(1,2), \underset{=P_1(1,1)+P_{12}}{\underset{j\in{\{1,3\}}}{\underbrace{min}}}(P_1(1,3)+P_{32})) = min(4,6+9) = 4
$$

$$
P_2(1,3)=min(P_1(1,3), \underset{j\in{\{1,4\}}}{\underbrace{min}}(P_1(1,4)+P_{43})) = min(6,\inf+1) = 6
$$

$A =$ Ensemble des arcs

$P_k(1,i)$=longueur minimale du chemin depuis le sommet 1 jusq'au sommet i en passant par au plus k arcs.

$$
\begin{array}{l}
init\ k=1 \\
P_1(1,i)= \end{array}\left\{
\begin{array}{l}
P_{1i} (poids de l'arc (1,i), \\
si \in Predecesseur(i) (ou (1,i) \in A)) \\
\end{array}
\right.
$$

$$
k = 2 \\
P_2(1,2) = min(\underset{=4}{P_1(1,2)})
$$


## Algorithme de Bellman ($O(n^2)$)

<!-- a -->
### etape 1
$P_1(1,1)=0$<br>
$P_1(1,2)=P_{12}=4$<br>
$P_1(1,3)=P_{13}=6$<br>
$P_1(1,4)=P_1(1,5)=P_1{(1,6)}=\inf$<br><br>
### etape 2
$h=2$<br>
$P_2(1,1) = 0$<br>
$P_2(1,2)=min(
    \underset{=4}{P_1(1,3)}, \underset{pred(2)=\{1,3\}}{\underbrace{min}}(
    \underset{=4}{\underbrace{\underset{\underset{=0}{}}{P_1(1,1)+\underset{=4}{P_{12}}}}})) = min(6,\inf+1
) = 6 $<br>
### etape 3
$P_3(1,1)=0$<br>
$P_3(1,2)=min(
    4, \underset{
            pred(2)=\{1,3\}
        }{
            \underbrace{min(\ \ \ \ \ \ \ \ \ \ \ \ \ \ )}
        }
) = 4 $<br>
$P_3(1,3)=min(
    6, \underset{
        \underbrace{pred(3)=\{1,4\}}
    }{
        min(6,P_2(1,4)+P_{43})
    }
) = min(6, 6,7+1) = 6$<br><br>
$P_3(1,4)=min(7,\underset{pred(4)=\{2\}}{\underbrace{min}}(P_2(1,2)+P_{14}))$<br>
### etape 4

a écrire

## Algorithme de Floyd-Warshall ($O(|n|^3)$)

<hr>

algorithme : <br><br>
$pour k=1,...,n$<br>
$\ \ \ \ pour\ i=1,...,n$<br>
$\ \ \ \ \ \ \ \ pour\ j=1,...,n$<br>
$\ \ \ \ \ \ \ \ \ \ \ \ C_k(i,j)=min(C_{k-1}(i,j),C_{k-1}(i,j)+C_{k-1}(k_{ij}))$<br>
$\ \ \ \ \ \ \ \ endp$<br>
$\ \ \ \ endp$<br>
$endp$<br>
<hr>


$
L = 
    \begin{matrix}
    \ \\ 1 \\ 2 \\ 3 \\ 4 \\ 5 \\ 6
    \end{matrix}    
    \begin{pmatrix}
    1 & 2 & 3 & 4 & 5 & 6 \\
    0 & 4 & 6 & \infty & \infty & \infty \\
    \infty & 0 & \infty & 3 & \infty & \infty \\
    \infty & 9 & 0 & \infty & 4 & \infty \\
    \infty & \infty & 1 & 0 & 1 & 4 \\
    \infty & \infty & \infty & \infty & 0 & 2 \\
    \infty & \infty & \infty & \infty & \infty & 0
    \end{pmatrix}
$


solution : 
```
M =[
    [0,4,6,inf,inf,inf]
    [0,4,6,7,10,inf],
    [0,4,6,7,8,11],
    [0,4,6,7,8,10],
    [0,4,6,7,8,10] <-- solution
]
```